In [94]:
from bank import Bank
import pandas as pd
import os
import re

In [2]:
!python --version

Python 3.6.12


In [3]:
os.environ.update({i[0]: i[1] for i in [i.split('=') for i in open('.env').read().split()]})

In [4]:
trs = Bank.transactions()

In [5]:
dft = pd.DataFrame(trs)

In [46]:
dft['remittanceInformationUnstructured'].iloc[3]

'PAGO CON TARJETA EN HOGAR, MUEBLES, DECORACION Y ELECTR // PAGO CON TARJETA // IKEA SEVILLA             CASTILLEJA DEES'

In [156]:
dft['accountId'].unique()

array(['ecd4061f-fca6-487e-960b-31d0e07dd1a4',
       'e4b0d95e-ddb1-4655-92e1-d671616cf674'], dtype=object)

In [158]:
id_iban = {'ecd4061f-fca6-487e-960b-31d0e07dd1a4': 'ES4501821252340201668917', 
           'e4b0d95e-ddb1-4655-92e1-d671616cf674': 'ES1601821252330201679188'}

In [173]:
import openpyxl
from pathlib import Path

In [272]:
xlsx_file = Path('./data/8917.xlsx')

In [273]:
wb_obj = openpyxl.load_workbook(xlsx_file) 


In [274]:
col_range = list(range(2, 12))
col_row = 5

In [275]:
col_names = {wb_obj.active.cell(col_row,i).value:i for i in col_range}

In [276]:
data = {k: [wb_obj.active.cell(i,v).value for i in range(col_row+1, 10**4)] for k, v in col_names.items()}

In [277]:
df = pd.DataFrame(data)

In [278]:
df = df[~df['IBAN'].isna()]

In [279]:
df['Fecha'] = (df['Fecha'].dt.date).astype(str)

In [280]:
df['F.Valor'] = (df['F.Valor'].dt.date).astype(str)

In [283]:
df.to_csv('./data/transactions_8917.hsv', sep='^', index=False)

In [285]:
#pd.read_csv('./data/transactions_8917.hsv',sep='^')

In [286]:
xlsx_file = Path('./data/9188.xlsx')
wb_obj = openpyxl.load_workbook(xlsx_file) 


In [287]:
data = {k: [wb_obj.active.cell(i,v).value for i in range(col_row+1, 10**4)] for k, v in col_names.items()}
df = pd.DataFrame(data)

In [288]:
df = df[~df['IBAN'].isna()]

In [289]:
df['Fecha'] = (df['Fecha'].dt.date).astype(str)
df['F.Valor'] = (df['F.Valor'].dt.date).astype(str)

In [290]:
df.to_csv('./data/transactions_9188.hsv', sep='^', index=False)

In [294]:
df = pd.concat([pd.read_csv(i, sep='^') for i in ['./data/transactions_9188.hsv', './data/transactions_8917.hsv']])

In [298]:
df.rename(columns={'Fecha':'bookingDate',
                   'F.Valor': 'valueDate', 
                   'Importe': 'transactionAmount',
                   'Disponible' : 'balance', 
                   'Divisa': 'transactionCurrency'}, inplace=True)

In [300]:
iban_id = {'ES4501821252340201668917':'ecd4061f-fca6-487e-960b-31d0e07dd1a4', 
           'ES1601821252330201679188': 'e4b0d95e-ddb1-4655-92e1-d671616cf674'}

In [305]:
df['accountId'] = df['IBAN'].map(iban_id)

In [312]:
df['remittanceInformationUnstructured'] = df['Movimiento'].str.upper() + ' EN ' + df['Concepto'].str.upper() + ' // ' + df['Observaciones'].str.upper()

In [318]:
df = df.drop(['Movimiento', 'Concepto', 'Observaciones'], axis=1)

In [332]:
df.to_csv('./data/merged.hsv', sep='^', index=False )